# Lesson 1 Exercise 2: Creating a Table with Apache Cassandra
<img src="images/cassandralogo.png" width="250" height="250">

### Walk through the basics of Apache Cassandra. Complete the following tasks:<li> Create a table in Apache Cassandra, <li> Insert rows of data,<li> Run a simple SQL query to validate the information. <br>
`#####` denotes where the code needs to be completed.
    
Note: __Do not__ click the blue Preview button in the lower taskbar

#### Import Apache Cassandra python package

In [6]:
import cassandra

In [7]:
#
# Copyright Amazon.com, Inc. or its affiliates. All Rights Reserved.
# SPDX-License-Identifier: MIT-0
#
try:
    import os
    import boto3
    import ssl
    import sys
    from boto3 import Session
    from cassandra_sigv4.auth import AuthProvider, Authenticator, SigV4AuthProvider
    from ssl import SSLContext, PROTOCOL_TLSv1_2, CERT_REQUIRED
    from cassandra.cluster import Cluster
    from cassandra import ConsistencyLevel
    from cassandra.query import SimpleStatement
except ImportError:
    raise RuntimeError('Required packages Failed To install please run "python Setup.py install" command or install '
                       'using pip')


### Create a connection to the database

In [8]:
ssl_context = SSLContext(PROTOCOL_TLSv1_2)
cert_path = os.path.join(os.getcwd(), 'resources/sf-class2-root.crt')
ssl_context.load_verify_locations(cert_path)
ssl_context.verify_mode = CERT_REQUIRED

# this will automatically pull the credentials from either the
# ~/.aws/credentials file
# ~/.aws/config 
# or from the boto environment variables.
boto_session = boto3.Session()


# verify that the session is set correctly
credentials = boto_session.get_credentials()

if not credentials or not credentials.access_key:
    sys.exit("No access key found, please setup credentials (AWS_ACCESS_KEY_ID, AWS_SECRET_ACCESS_KEY) according to https://docs.aws.amazon.com/cli/latest/userguide/cli-configure-quickstart.html#cli-configure-quickstart-precedence\n")


region = boto_session.region_name

if not region:  
    sys.exit("You do not have a region set.  Set environment variable AWS_REGION or provide a configuration see https://docs.aws.amazon.com/cli/latest/userguide/cli-configure-quickstart.html#cli-configure-quickstart-precedence\n")


auth_provider = SigV4AuthProvider(boto_session)
contact_point = "cassandra.{}.amazonaws.com".format(region)

cluster = Cluster([contact_point], 
                 ssl_context=ssl_context, 
                 auth_provider=auth_provider,
                 port=9142)

session = cluster.connect()
print("CONNECTION TO KEYSPACES SUCCESSFUL2")

rows = session.execute('select * from system_schema.keyspaces')
print("PRINTING SCHEMA INFORMATION2")
for r in rows.current_rows:  
    print("Found Keyspace: {}".format(r.keyspace_name))


 

CONNECTION TO KEYSPACES SUCCESSFUL2
PRINTING SCHEMA INFORMATION2
Found Keyspace: system_schema
Found Keyspace: system_schema_mcs
Found Keyspace: system
Found Keyspace: my_demo_keyspace
Found Keyspace: newkeyspace
Found Keyspace: test


### TO-DO: Create a keyspace to do the work in 

In [9]:
## TO-DO: Create the keyspace
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS song_library 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

### TO-DO: Connect to the Keyspace

In [10]:
## To-Do: Add in the keyspace you created
try:
    session.set_keyspace('song_library')
except Exception as e:
    print(e)

### Create a Song Library that contains a list of songs, including the song name, artist name, year, album it was from, and if it was a single. 

`song_title
artist_name
year
album_name
single`

### TO-DO: You need to create a table to be able to run the following query: 
`select * from songs WHERE year=1970 AND artist_name="The Beatles"`

In [26]:
## TO-DO: Complete the query below
query = """CREATE TABLE IF NOT EXISTS song_library (song_title text, artist_name text, year int, album_name text, single boolean, PRIMARY KEY ((year, artist_name),album_name ) ) """

try:
    session.execute(query)
except Exception as e:
    print(e)


### TO-DO: Insert the following two rows in your table
`First Row:  "Across The Universe", "The Beatles", "1970", "False", "Let It Be"`

`Second Row: "The Beatles", "Think For Yourself", "False", "1965", "Rubber Soul"`

In [27]:
## Add in query and then run the insert statement

query = SimpleStatement("INSERT INTO song_library (song_title , artist_name , year , album_name , single ) VALUES (%s,%s,%s,%s,%s)",
    consistency_level=ConsistencyLevel.LOCAL_QUORUM)


try:
    session.execute(query, ("Let It Be", "The Beatles", 1970, "Across The Universe" , False))
except Exception as e:
    print(e)  
    
try:
    session.execute(query, ("Think For Yourself", "The Beatles", 1965, "Rubber Soul", False))
except Exception as e:
    print(e)  
    

### TO-DO: Validate your data was inserted into the table.

In [35]:
## TO-DO: Complete and then run the select statement to validate the data was inserted into the table
query = 'SELECT * FROM song_library'
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.song_title , row.artist_name , row.year , row.album_name , row.single)

Let It Be The Beatles 1970 Across The Universe False
Think For Yourself The Beatles 1965 Rubber Soul False


### TO-DO: Validate the Data Model with the original query.

`select * from songs WHERE YEAR=1970 AND artist_name="The Beatles"`

In [37]:
##TO-DO: Complete the select statement to run the query 
query = "select * from song_library WHERE YEAR=1970 AND artist_name='The Beatles' "
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.song_title , row.artist_name , row.year , row.album_name , row.single)

Let It Be The Beatles 1970 Across The Universe False


### And Finally close the session and cluster connection

In [38]:
session.shutdown()
cluster.shutdown()